
Prepared by Otmane Ainelkitane
- Email: otmaneaine@gmail.com
- LinkedIn: [Otmane Ainelkitane](https://www.linkedin.com/in/otmane-ainelkitane/)


# The Role of Quantum Key Distribution in Secure Communication

## Introduction
Secure communication is vital in today's digital age, where sensitive information is transmitted over various networks. Encryption plays a crucial role in safeguarding this information from unauthorized access. However, traditional encryption methods face challenges from advances in computing power and potential vulnerabilities in cryptographic algorithms. Quantum Key Distribution (QKD) offers a promising solution to address these challenges by leveraging the principles of quantum mechanics to establish secure keys between communicating parties. We will discuss the development of encryption and decryption functions using QKD, analyze the impact of an eavesdropper on key development, and explore the importance of QKD and other quantum encryption algorithms for future secure communication.

## Development of Encryption and Decryption Functions
The encryption and decryption functions implemented in this project utilize classical XOR encryption, augmented by a shared key generated through QKD.

This method is refer to as Vernam cipher[2],(private key cryptographic system), sometimes called one-time-pad. Its main idea is that Alice and Bob share prior to the communication of the message, a private key, the secrecy of this key is crucial in these cryptographic systems, this can be fulfilled by confidential personal meeting between the two parties or delivered through trusted couriers or private secured communication links.
 
The encryption function takes the original message and the shared key as input, iterates through each bit of the message, and performs a bitwise XOR operation with the corresponding bit of the key. The result is the encrypted message. The decryption function reverses this process, using the shared key to decrypt the encrypted message and recover the original plaintext(message).


In [ ]:
# import all necessary libraries
import cirq
import random

In [ ]:
# the message that Alice will communicate to Bob
message = "We are out of pizza"
# We need first to convert the message to its binary format
# Get bytes of the message using UTF-8 encoding
bytes_message = message.encode('utf-8')

# Convert each byte to a binary string with leading zeros for 8 bits
binary_message = ''.join(format(b, "08b") for b in bytes_message)
# the length of binary_message
n = len(binary_message)

# QKD part
Initial_key_lenght = int(4.3*n) # we will use 4.3 times the length of the binary_message as a safety measure
Alice_qubits = cirq.LineQubit.range(Initial_key_lenght) # quantum register with Initial_key_lenght qubits
# Quantum circuit for Alice state
Alice_circuit = cirq.Circuit()

Alice_initial_key=[] #Initial bit string to send
Alice_basis=[] #List to save information about encoding basis
Bob_basis=[] #List to save information about decoding basis

#Creating random bit string
binary_list = [0,1]
equal_weights = [0.5, 0.5]
# Alice chooses randomly a string of binary bits
Alice_initial_key = random.choices(binary_list, weights=equal_weights, k=Initial_key_lenght)
    
#Preparing qubits, apply X gate if bit is equal 1
for i, bit in enumerate(Alice_initial_key):
    if bit==1:
        Alice_circuit.append(cirq.X(Alice_qubits[i])) # apply x-gate

#Encoding
Encoding_basis = ['Z', 'X']
Alice_basis = random.choices(Encoding_basis, weights=equal_weights, k=Initial_key_lenght) #Alice randomly pick a string of basis 
for i, basis in enumerate(Alice_basis):
    if basis=='Z': #if basis is 'Z' , then she encodes the bit in Z basis(do nothing)
        pass
    else: #if basis is 'X' , then she encodes the bit in X basis
        Alice_circuit.append(cirq.H(Alice_qubits[i]))

print("Alice\'s initial key ", Alice_initial_key)
print('\nAlice\'s randmoly chosen bases: ', Alice_basis)
#print('\nAlice\'s circuit:\n', Alice_circuit)



In [ ]:
# Decoding
# Quantum circuit for Bob's state
Bob_circuit = cirq.Circuit()

Bob_basis = random.choices(Encoding_basis, weights=equal_weights, k=Initial_key_lenght) #Bob randomly pick a string of basis
# Based on the randomly chosen basis, Bob apply that gate so can be able to measure in the appropriate basis
for i, basis in enumerate(Bob_basis):
    if basis=='Z': #if basis is 'Z'
        pass 
    else: ##if basis is 'X'
        Bob_circuit.append(cirq.H(Alice_qubits[i]))
        

# Bob measures the recieving qubits        
Bob_circuit.append(cirq.measure(*Alice_qubits, key='Bob key'))
# the whole circuit
BB84_circuit = Alice_circuit + Bob_circuit

# simulating the BB84 protocol  
n_rep = 100
sim = cirq.Simulator()
results = sim.run(BB84_circuit, repetitions=n_rep)
Bob_initial_key = results.measurements['Bob key'][0]



print("Alice sent:", Alice_initial_key)
print("Alice encoding basis:", Alice_basis)
print("Bob received:", Bob_initial_key)
print("Bob decoding basis:", Bob_basis)

In [ ]:
#Sifting
Alice_final_key=[] #Alice list for matching basis
Bob_final_key=[] #Bob list for matching basis
for j in range(len(Alice_basis)): #Going through list of bases 
    if Alice_basis[j] == Bob_basis[j]: #Comparing
        #Keeping key bit if bases matched
        Alice_final_key.append(Alice_initial_key[j])
        Bob_final_key.append(Bob_initial_key[j]) 
    else:
        pass #Discard round if bases mismatched

print("Alice\'s key =", Alice_final_key)
print("Bob key\'s =", Bob_final_key)

We consider that the quantum channel used in this protocol is noiseless for simplicity, therefore the Quantum Bit Error Rate($QBER$) between Alice and Bob will only be caused by the eavesdropper Eve. Since we are simulating this protocol in the absence of Eve, we expect to have a $QBER = 0$, otherwise, we will have a $QBER > 0$.
In the latter case, Alice and Bob would agree on a specific thershold, that if the $QBER$ exceeds the threshold, they would discard the compromised key and initiate the QKD process again to establish a new secure key.


In [ ]:
#QBER
batch = len(Alice_final_key)//2    #To divide without remainer, use //
errors=0
for i in range(batch):
    bit_index = random.randrange(len(Alice_final_key)) 
    tested_bit = Alice_final_key[bit_index]
    print ("Alice randomly selected bit index =", bit_index, ", and its value is = ", tested_bit)
    if Alice_final_key[bit_index]!=Bob_final_key[bit_index]: #comparing tested rounds
        errors=errors+1 #calculating errors
    #removing tested bits from key strings
    del Alice_final_key[bit_index] #Use del to specify the index of the element you want to delete
    del Bob_final_key[bit_index]
QBER=errors/batch #calculating QBER
        
print("QBER value =", QBER)
print("Alice's secret key =", Alice_final_key)
print("Bob's secret key =", Bob_final_key)

The encryption and decryption steps typically occur on a classical channel after the successful completion of the QKD process. Once Alice and Bob have established a secure key through QKD, they can use this shared key to encrypt and decrypt their messages using classical cryptographic algorithms.

In [96]:
#Encryption(Alice)
# Use the generated shared key to encrypt the message using classical XOR encryption
def encrypt_message(message, key):
    encrypted_message = ''
    for i in range(len(message)):
        encrypted_bit = int(message[i]) ^ key[i]
        encrypted_message += str(encrypted_bit)
    return encrypted_message

encrypted_message = encrypt_message(binary_message, Alice_final_key)

In [97]:
# Decryption(Bob)
def binary_to_str(binary_string, binary_length=8):
    # Convert binary string to bytes
    binary_bytes = bytes(int(binary_string[i:i+binary_length], 2) for i in range(0, len(binary_string), binary_length))
    text = binary_bytes.decode('utf-8')
    return text

# Use the shared key received from Alice to decrypt the message
def decrypt_message(encrypted_message, key):
    decrypted_message = ''
    for i in range(len(encrypted_message)):
        decrypted_bit = int(encrypted_message[i]) ^ key[i]
        decrypted_message += str(decrypted_bit)
    return decrypted_message

decrypted_message = decrypt_message(encrypted_message, Bob_final_key)
# we will convert the binary message that Bob received into a string using the funtion we have built above.
decrypted_message = binary_to_str(decrypted_message)




In [98]:
#  Verification

# Verify if the decrypted message matches the original message
if decrypted_message == message:
    print("Encryption and decryption successful!")
    print("Original message:", message)
    print("Decrypted message:", decrypted_message)
else:
    print("Encryption and decryption failed!")

Encryption and decryption successful!
Original message: We are out of pizza
Decrypted message: We are out of pizza


## Impact of an Eavesdropper on Key Development
In the process of Quantum Key Distribution (QKD), Alice and Bob exchange quantum states to establish a shared key while monitoring for any interception attempts by an eavesdropper, traditionally referred to as Eve. However, if an eavesdropper intercepts the quantum states during key development, they will unavoidably alter the quantum states due to the principles of quantum mechanics.

Under the assumption that Alice and Bob use a noiseless quantum channel, they can calculate the Quantum Bit Error Rate (QBER) to determine the existence of a potential eavesdropper after the sifting phase. During this phase, Alice and Bob discard the mismatching measurements (those in which they used different bases). If the QBER is greater than 0, then Eve is detected.

It's important to note that in reality, all quantum channels suffer from noise. Therefore, Alice and Bob must account for this noise when calculating the QBER, as some quantum states will be altered even in the absence of an eavesdropper. This means that even without interference from Eve, the quantum channel noise can contribute to a non-zero QBER.

Eve can exploit this quantum channel noise to her advantage. By intercepting the quantum states, she can gain partial information about the key without being detected. This compromises the security of the communication channel, as Eve could potentially decrypt the encrypted messages exchanged between Alice and Bob.

## Importance of QKD and Quantum Encryption Algorithms
The development of QKD and other quantum encryption algorithms is essential for ensuring the security of future communication systems. Unlike classical encryption methods, which rely on the difficulty of solving mathematical problems, quantum encryption leverages the fundamental principles of quantum mechanics, offering inherent security against eavesdropping attacks. QKD enables the generation of secure keys that are immune to interception attempts by eavesdroppers, making it a crucial component of quantum-safe communication protocols. Furthermore, the continuous advancement of quantum technologies holds the promise of achieving even more secure and efficient encryption schemes, paving the way for the future of secure communication.

### Applications:

1. **Banking and Financial Services:**
   - QKD enhances the security of financial transactions, protecting sensitive information such as banking credentials, credit card numbers, and financial transactions from eavesdropping attacks.
   - Banks and financial institutions can use QKD to establish secure communication channels between branches or with customers, ensuring the confidentiality and integrity of financial data.

2. **Healthcare and Medical Records:**
   - In healthcare, patient records and medical data contain highly sensitive information that must be protected from unauthorized access.
   - QKD can be used to encrypt medical records and ensure the privacy of patient information, preventing unauthorized access or tampering.

3. **Government and Military Communication:**
   - Government agencies and military organizations require highly secure communication channels to exchange classified information and strategic plans.

## Conclusion
In conclusion, Quantum Key Distribution and quantum encryption algorithms offer a robust solution to the challenges of secure communication in the digital era. By leveraging the principles of quantum mechanics, these technologies enable the generation of secure keys and encryption methods that are resilient against eavesdropping attacks. As the field of quantum computing and communication continues to advance, the development and deployment of QKD and other quantum encryption algorithms will play a vital role in ensuring the security and privacy of communication systems in the future.

## References
1. Charles H. Bennett, Gilles Brassard. Quantum cryptography: Public key distribution and coin tossing. Proceedings of IEEE International Conference on Computers, Systems and Signal Processing, 1984.
2. G. S. Vernam. Cipher Printing Telegraph Systems For Secret Wire and Radio Telegraphic Communications
Transactions of the American Institute of Electrical Engineers, 1926.
3. Preskill, J. (2018). Quantum Computing in the NISQ era and beyond. Quantum, 2, 79.